In [4]:
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np

from lib.dataset.hicodet_hake import HicoDetHake
from lib.dataset.vcoco import VCoco
from lib.dataset.cocoa import Cocoa
from lib.dataset.hico_cocoa import HicoCocoa

In [2]:
hh = HicoDetHake()
vcoco = VCoco()
cocoa = Cocoa()

In [2]:
# Generate V-COCO unseen actions. Note: this is deterministic, as 
# all unseen actions are used.

pacts = sorted({hh.get_action_from_state(x) for x in hh.states})
print(', '.join(sorted(pacts)), '\n')

hacts = [hh.actions[0]] + [a.split('_')[0] for a in hh.actions[1:]]
print(', '.join(sorted(hacts)), '\n')

v_to_h_mapping = {'lay': 'lie', 
                  'look': 'inspect',
                  'skateboard': 'ride', 
                  'ski': 'ride',
                  'snowboard': 'ride',
                  'surf': 'ride',
                  'work': 'type'
                 }
vacts = [vcoco.actions[0]] + [a.split('_')[0] for a in vcoco.actions[1:]]
print(', '.join(sorted(vacts)), '\n')
vacts = [v_to_h_mapping.get(a, a) for a in vacts]
print(', '.join(sorted(vacts)), '\n')

new_v = set(vacts) - set(hacts)
print(', '.join(sorted(new_v)), '\n')

words_in_pacts = {w for a in pacts for w in a.split('_') if a != hh.null_action}
print(', '.join(sorted(words_in_pacts)), '\n')

possible_unseen_acts_inds = np.array([i for i, a in enumerate(vacts) 
                                      if a != hh.null_action and a not in words_in_pacts])
possible_unseen_acts = [vcoco.actions[i] for i in possible_unseen_acts_inds]
print(', '.join(sorted(possible_unseen_acts)), '\n')
print(', '.join(sorted({'_'.join(a.split('_')[:-1]) for a in possible_unseen_acts})), '\n')

with open(f'zero-shot_inds/vcoco_seen_inds_0.pkl.push', 'wb') as f:  # Fake one to use for test (no actual unseen)
    pickle.dump({'train': {'act': np.arange(vcoco.num_actions)}},
                f)

with open(f'zero-shot_inds/vcoco_seen_inds_1.pkl.push', 'wb') as f:
    seen_acts = np.setdiff1d(np.arange(vcoco.num_actions), possible_unseen_acts_inds)
    pickle.dump({'train': {'act': seen_acts}},
                f)

__no_interaction__, be_close_to, be_close_with, blow, carry, catch_with_something, cut_with_something, dribble, drink_with, eat, feed, gesture_to, hold, hold_in_both_hands, hug, inspect, is_close_with, jump_down, jump_with, kick, kiss, lick, lift, pinch, point_to, point_with, pour_into, press, pull, pull_with_something, push, put_on, put_something_over_head, raise, reach_for, run_to, run_with, scratch, shoulder_carry, sit_beside, sit_in, sit_on, smell, squeeze, stand_on, straddle, swing, talk_to, talk_with_something, throw, throw_out, touch, tread_on, twist, use_something_to_point_to, walk_away, walk_to, walk_with, wash, wash_with_something, wear, write_on 

__no_interaction__, adjust, assemble, block, blow, board, break, brush, buy, carry, catch, chase, check, clean, control, cook, cut, cut, direct, drag, dribble, drink, drive, dry, eat, eat, exit, feed, fill, flip, flush, fly, greet, grind, groom, herd, hit, hold, hop, hose, hug, hunt, inspect, install, jump, kick, kiss, lasso, launc

In [9]:
# Check seen classes file

ds = HicoCocoa()
name = 'hc'
fnum = 1

d = pickle.load(open(f'zero-shot_inds/{name}_seen_inds_{fnum}.pkl.push', 'rb'))

try:
    ainds = d['train']['act']
    oinds = d['train'].get('obj', np.arange(ds.num_objects))

    print(f'{len(set(ainds.tolist()))} / {ds.num_actions} = '
          f'{len(set(ainds.tolist())) / ds.num_actions}')
    print(f'{len(set(oinds.tolist()))} / {ds.num_objects} = '
          f'{len(set(oinds.tolist())) / ds.num_objects}')

    fg_hois = [(a, o) for a, o in ds.interactions if a > 0]
    print(f'{len(fg_hois)} / {ds.num_interactions} = '
          f'{len(fg_hois) / ds.num_interactions}')

    seen_hois_inds = [i for i, (a, o) in enumerate(ds.interactions)
                      if a in ainds]
except KeyError:
    seen_hois_inds = d['train']['hoi']

seen_hois = [ds.interactions[i] for i in seen_hois_inds]
print(f'{len(seen_hois)} / {ds.num_interactions} = '
      f'{len(seen_hois) / ds.num_interactions}')
seen_fg_hois = [ds.interactions[i] for i in seen_hois_inds
                if ds.interactions[i, 0] > 0]
print(f'{len(seen_fg_hois)} / {ds.num_interactions} = '
      f'{len(seen_fg_hois) / ds.num_interactions}')

unseen_hois_inds = np.setdiff1d(np.arange(ds.num_interactions), seen_hois_inds)
unseen_fg_hois = [ds.interactions[i] for i in unseen_hois_inds
                if ds.interactions[i, 0] > 0]
print(f'{len(unseen_fg_hois)} / {ds.num_interactions} = '
      f'{len(unseen_fg_hois) / ds.num_interactions}')

600 / 2071 = 0.28971511347175277
520 / 2071 = 0.2510864316755191
1471 / 2071 = 0.7102848865282472


In [21]:
# Generate COCO-A unseen actions. Note: this is deterministic, as all unseen actions are used.

unseen_acts_str = set(cocoa.actions) - set(hh.actions[1:])
interactions = [(a, o) for (a, o) in cocoa.interactions if a > 0]
oov_interactions = [(a, o) for (a, o) in interactions if cocoa.actions[a] in unseen_acts_str]
print(len(interactions), len(oov_interactions))

oov_acts = {a for a, o in oov_interactions}
oov_acts_str = {cocoa.actions[a] for a in oov_acts}
print(cocoa.num_actions, len(unseen_acts_str), len(oov_acts), 
      len(oov_acts_str - unseen_acts_str), len(unseen_acts_str - oov_acts_str))

seen_act_inds = np.array(sorted(set(range(cocoa.num_actions)) - oov_acts))
print(cocoa.num_actions, len(oov_acts), seen_act_inds.size)
assert 0 in seen_act_inds

with open(f'zero-shot_inds/cocoa_seen_inds_0.pkl.push', 'wb') as f:  # Fake one to use for test (no actual unseen)
    pickle.dump({'train': {'act': np.arange(cocoa.num_actions)}},
                f)

with open(f'zero-shot_inds/cocoa_seen_inds_1.pkl.push', 'wb') as f:
    pickle.dump({'train': {'act': seen_act_inds}},
                f)

1681 1048
146 104 83 0 21
146 83 63


In [6]:

hc = HicoCocoa()

seen_interactions = hc.hico_interaction_mapping

assert [hc.interactions_str[i] for i in seen_interactions] == hc.hico.interactions_str

with open(f'zero-shot_inds/hc_seen_inds_0.pkl.push', 'wb') as f:  # Fake one to use for test (no actual unseen)
    pickle.dump({'train': {'hoi': np.arange(hc.num_interactions)}},
                f)
with open(f'zero-shot_inds/hc_seen_inds_1.pkl.push', 'wb') as f:
    pickle.dump({'train': {'hoi': seen_interactions}},
                f)
print('Done.')

Done.
